In [4]:
import cv2
import os
import mediapipe as mp
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.utils import to_categorical

In [10]:
# Preprocessing functions
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()
mp_drawing = mp.solutions.drawing_utils

In [7]:
def preprocess_frame(frame):
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            landmarks = [[lm.x, lm.y, lm.z] for lm in hand_landmarks.landmark]
            return landmarks
    return None

In [12]:
def load_data(data_path='data'):
    X, y = [], []
    labels = os.listdir(data_path)
    print(f"Labels found: {labels}")  # Debugging statement
    for label in labels:
        label_path = os.path.join(data_path, label)
        for img_file in os.listdir(label_path):
            img_path = os.path.join(label_path, img_file)
            frame = cv2.imread(img_path)
            landmarks = preprocess_frame(frame)
            if landmarks:
                X.append(landmarks)
                y.append(labels.index(label))
    return np.array(X), np.array(y)

In [13]:
# Load and preprocess the data
X_train, y_train = load_data()

Labels found: []


In [14]:
# Debugging: Check if data is loaded correctly
print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"Sample y_train: {y_train[:5]}")

X_train shape: (0,)
y_train shape: (0,)
Sample y_train: []


In [15]:
# Check if data arrays are not empty
if X_train.size == 0 or y_train.size == 0:
    raise ValueError("No data loaded. Please check the data directory and preprocessing steps.")

ValueError: No data loaded. Please check the data directory and preprocessing steps.

In [ ]:
# Step 3: Model Training
num_classes = y_train.shape[1]

In [17]:
model = Sequential([
    Flatten(input_shape=(21, 3)),  # 21 landmarks with 3 coordinates (x, y, z)
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')
])

In [18]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, validation_split=0.2)

ValueError: Training data contains 0 samples, which is not sufficient to split it into a validation and training set as specified by `validation_split=0.2`. Either provide more data, or a different value for the `validation_split` argument.

In [8]:
# Step 4: Real-Time Inference
def real_time_inference(model):
    cap = cv2.VideoCapture(0)
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        landmarks = preprocess_frame(frame)
        if landmarks:
            landmarks = np.array(landmarks).reshape(1, 21, 3)
            prediction = model.predict(landmarks)
            predicted_label = np.argmax(prediction)
            print(f"Predicted sign: {predicted_label}")
        
        cv2.imshow("Frame", frame)
        key = cv2.waitKey(0)
        if key == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

In [6]:
real_time_inference(7)

NameError: name 'preprocess_frame' is not defined